# Matchmaker Basic Usage

This notebook demonstrates the core matchmaker library functionality:
- Loading interaction data with gender support
- Computing engagement scores
- Computing popularity metrics and leagues
- Training ALS models for recommendations

### ⚠️ Note: NVIDIA Driver Installation Required ⚠️

Verify with: ```nvidia-smi```

In [ ]:
from matchmaker import matchmaker
import cudf

In [ ]:
raw = cudf.read_csv("data/swipes_clean.csv")

In [ ]:
engine = matchmaker.MatchingEngine()

In [ ]:
engine.load_interactions("data/swipes_clean.csv", 
  decider_col='decidermemberid',
  other_col='othermemberid', 
  like_col='like', 
  timestamp_col='timestamp',
  gender_col='decidergender')

In [ ]:
engine.run_engagement()

In [ ]:
engine.run_popularity()

In [ ]:
engine.build_recommender()

In [ ]:
user_df = engine.user_df
interaction_df = engine.interaction_df
interaction_graph = engine.interaction_graph
als_model = engine.als_model

---

In [ ]:
# Test batch performance
import time
t = user_df.dropna()
t = t[t.gender=='M'].sample(20_000).user_id.to_arrow().to_pylist()

start = time.time()
results = engine.recommend_batch(t, k=200)
elapsed = time.time() - start

print(f"✅ Batch recommended for {len(t)} users in {elapsed:.2f} seconds")
print(f"   ({elapsed*1000/len(t):.1f} ms per user)")
print(f"   Sample result for user {t[0]}: {len(results[t[0]])} recommendations")